<a href="https://colab.research.google.com/github/WZRP/sd-minference/blob/main/SD-Minference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title # Install Dependencies
import os
import zipfile
import shutil
import time
from subprocess import getoutput
from IPython.utils import capture
from google.colab import drive

%store -r

# root_dir
root_dir = "/content"
repo_dir = os.path.join(root_dir, "ai-image")
model_dir = os.path.join(root_dir, "model_dir")
vae_dir = os.path.join(root_dir, "vae")

# repo_dir
accelerate_config = os.path.join(repo_dir, "accelerate_config/config.yaml")
tools_dir = os.path.join(repo_dir, "tools")

for store in [
    "root_dir",
    "repo_dir",
    "model_dir",
    "vae_dir",
    "accelerate_config",
    "tools_dir",
]:
    with capture.capture_output() as cap:
        %store {store}
        del cap

repo_url = "https://github.com/WZRP/sd-minference"
bitsandytes_main_py = "/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py"
branch = ""  # @param {type: "string"}
install_xformers = True  # @param {'type':'boolean'}
mount_drive = False  # @param {type: "boolean"}
verbose = False # @param {type: "boolean"}

def read_file(filename):
    with open(filename, "r") as f:
        contents = f.read()
    return contents

def write_file(filename, contents):
    with open(filename, "w") as f:
        f.write(contents)

def clone_repo(url):
    if not os.path.exists(repo_dir):
        os.chdir(root_dir)
        !git clone {url} {repo_dir}
    else:
        os.chdir(repo_dir)
        !git pull origin {branch} if branch else !git pull

def install_dependencies():
    s = getoutput('nvidia-smi')

    if 'T4' in s:
        !sed -i "s@cpu@cuda@" library/model_util.py

    !pip install torchmetrics --upgrade-strategy only-if-needed --no-deps
    !pip install lightning-utilities --upgrade-strategy only-if-needed --no-deps
    !pip install tokenizers --upgrade-strategy only-if-needed --no-deps
    !pip install {'-q' if not verbose else ''} --upgrade-strategy only-if-needed --no-deps -r requirements.txt
    !pip install {'-q' if not verbose else ''} --upgrade-strategy only-if-needed --no-deps torch torchvision torchaudio torchtext torchdata --extra-index-url https://download.pytorch.org/whl/cu118
    if install_xformers:
        !pip install {'-q' if not verbose else ''} --upgrade-strategy only-if-needed --no-deps xformers triton

    from accelerate.utils import write_basic_config

    if not os.path.exists(accelerate_config):
        write_basic_config(save_location=accelerate_config)

def remove_bitsandbytes_message(filename):
    welcome_message = """
def evaluate_cuda_setup():
    print('')
    print('='*35 + 'BUG REPORT' + '='*35)
    print('Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues')
    print('For effortless bug reporting copy-paste your error into this form: https://docs.google.com/forms/d/e/1FAIpQLScPB8emS3Thkp66nvqwmjTEgxp8Y9ufuWTzFyr9kJ5AoI47dQ/viewform?usp=sf_link')
    print('='*80)"""

    new_welcome_message = """
def evaluate_cuda_setup():
    import os
    if 'BITSANDBYTES_NOWELCOME' not in os.environ or str(os.environ['BITSANDBYTES_NOWELCOME']) == '0':
        print('')
        print('=' * 35 + 'BUG REPORT' + '=' * 35)
        print('Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues')
        print('For effortless bug reporting copy-paste your error into this form: https://docs.google.com/forms/d/e/1FAIpQLScPB8emS3Thkp66nvqwmjTEgxp8Y9ufuWTzFyr9kJ5AoI47dQ/viewform?usp=sf_link')
        print('To hide this message, set the BITSANDBYTES_NOWELCOME variable like so: export BITSANDBYTES_NOWELCOME=1')
        print('=' * 80)"""

    contents = read_file(filename)
    new_contents = contents.replace(welcome_message, new_welcome_message)
    write_file(filename, new_contents)

def main():
    os.chdir(root_dir)

    if mount_drive:
        if not os.path.exists("/content/drive"):
            drive.mount("/content/drive")

    for dir in [
        vae_dir
    ]:
        os.makedirs(dir, exist_ok=True)

    clone_repo(repo_url)

    if branch:
        os.chdir(repo_dir)
        status = os.system(f"git checkout {branch}")
        if status != 0:
            raise Exception("Failed to checkout branch or commit")

    os.chdir(repo_dir)

    !apt install aria2 {'-qq' if not verbose else ''}

    install_dependencies()
    time.sleep(3)

    remove_bitsandbytes_message(bitsandytes_main_py)

    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
    os.environ["BITSANDBYTES_NOWELCOME"] = "1"
    os.environ["SAFETENSORS_FAST_GPU"] = "1"

    cuda_path = "/usr/local/cuda-11.8/targets/x86_64-linux/lib/"
    ld_library_path = os.environ.get("LD_LIBRARY_PATH", "")
    os.environ["LD_LIBRARY_PATH"] = f"{ld_library_path}:{cuda_path}"

main()


In [ ]:
# @title # Install Models & Styles
import os

%store -r

os.chdir(root_dir)

models = {
    "Anime.ckpt": "https://huggingface.co/OpenAppMKT/Models/resolve/main/Anime.ckpt",
    "Flexible.safetensors": "https://huggingface.co/OpenAppMKT/Models/resolve/main/Flexible.safetensors",
    "Chillout.safetensors": "https://huggingface.co/OpenAppMKT/Models/resolve/main/Chillout.safetensors",
    "Realism.safetensors": "https://huggingface.co/OpenAppMKT/Models/resolve/main/Realism.safetensors",
    "Pixel.safetensors": "https://huggingface.co/OpenAppMKT/Models/resolve/main/Pixel.safetensors",
}

install_models = []

for model_name, model_url in models.items():
    install_models.append((model_name, model_url))

def install(model_name, url):
    base_name = os.path.basename(url)
    !aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -d {model_dir} -o {base_name} "{url}"

def install_model():
    for model in install_models:
        install(model[0], model[1])

install_model()

vae_dir = os.path.join(root_dir, "vae")
os.makedirs(vae_dir, exist_ok=True)

vae_url = "https://huggingface.co/OpenAppMKT/Models/resolve/main/anime.vae.pt"
vae_name = "anime.vae.pt"

if vae_url:
    !aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -d {vae_dir} -o {vae_name} "{vae_url}"

In [ ]:
# @title # Generate Images
%store -r

v2 = False
v_parameterization = False
models = {
    "Anime": "/content/model_dir/Anime.ckpt",
    "Fantasy": "/content/model_dir/Flexible.safetensors",
    "Portrait": "/content/model_dir/Chillout.safetensors",
    "Realism": "/content/model_dir/Realism.safetensors",
}
model = "Anime"  # @param ["Anime", "Fantasy", "Portrait", "Realism"]
model_directory = models.get(model)
prompt = ""  # @param {type: "string"}
negative = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"  # @param {type: "string"}
vae = "/content/vae/anime.vae.pt"
outdir = "/content/output_dir"
scale = 7  # param {type: "slider", min: 1, max: 40}
sampler = "k_dpm_2"
steps = 30  # param {type: "slider", min: 1, max: 100}
precision = "fp16"
width = "512" #@param ["512", "768", "1024"] {allow-input: false}
height = "768" #@param ["512", "768", "1024"] {allow-input: false}
style = "None" #@param ["None", "Pixel"] {allow-input: false}
pixel_style = "/content/model_dir/Pixel.safetensors"
images_per_prompt = 1
batch_size = 1
clip_skip = 2
seed = -1

final_prompt = f"{prompt} --n {negative}"

config = {
    "v2": v2,
    "v_parameterization": v_parameterization,
    "ckpt": model_directory,
    "outdir": outdir,
    "xformers": True,
    "vae": vae if vae else None,
    "fp16": True,
    "W": width,
    "H": height,
    "seed": seed if seed > 0 else None,
    "scale": scale,
    "sampler": sampler,
    "steps": steps,
    "max_embeddings_multiples": 3,
    "batch_size": batch_size,
    "images_per_prompt": images_per_prompt,
    "clip_skip": clip_skip if not v2 else None,
    "prompt": final_prompt,
    "network_module": "networks.lora" if style != "None" else None,
    "network_weight": pixel_style if style == "Pixel" else None,
    "network_mul": 0.8 if style == "Pixel" else None,
}

args = ""
for k, v in config.items():
    if isinstance(v, str):
        args += f'--{k}="{v}" '
    if isinstance(v, bool) and v:
        args += f"--{k} "
    if isinstance(v, float) and not isinstance(v, bool):
        args += f"--{k}={v} "
    if isinstance(v, int) and not isinstance(v, bool):
        args += f"--{k}={v} "

final_args = f"python gen_img_diffusers.py {args}"

os.chdir(repo_dir)
!{final_args}

from PIL import Image

def upscale_image_nearest(image_path):
    input_image = Image.open(image_path)
    width, height = input_image.size
    output_image = Image.new('RGB', (width*8, height*8))
    for x in range(width*8):
        for y in range(height*8):
            orig_x = x // 8
            orig_y = y // 8
            color = input_image.getpixel((orig_x, orig_y))
            output_image.putpixel((x, y), color)
    return output_image

def style_pixel_pro():
    import os
    files = os.listdir(outdir)
    files = [file for file in files if file.lower().endswith('.png')] # filter the files by extension
    file_paths = [os.path.join(outdir, file) for file in files]
    file_paths.sort(key=os.path.getmtime, reverse=True)
    latest_file_path = file_paths[0]
    !python /content/ai-image/postprocessor/pixeldetector.py -i {latest_file_path} -o {latest_file_path} -p
    upscaled_image = upscale_image_nearest(latest_file_path)
    upscaled_image.save(latest_file_path)

if style == "Pixel":
    style_pixel_pro()